# KNN from Scratch

Using the KNN algorithm from scikit-learn is quite simple. When you use it, you don't even need to know how KNN works and classifies the data points. The goal should be to really understand what you are doing. This is the only way to make sure you understand when something doesn't go the way you expect or want it to. Many people remember things best when they implement them themselves. So, let's build our own K-Nearest-Neighbors classifier!

**The purpose of this notebook is to help you remember the steps necessary to classify samples with KNN.**

To test if your code works, you can use the Iris dataset as a data example.
Let's make a plan and break this big task into smaller steps!


1. What information and data does the algorithm need to train and predict the classes of new instances?
This will be the input for our function! 

2. calculate the distance between the test point and each existing data point in the training data.
3. determine the nearest k neighbors.
4. make predictions based on these neighbors.

You have already implemented a function to calculate the distance between points, which will now come in handy.

A good way to get started, is to ignore the syntax and just write in simple text what you want your program to do aka **write pseudo-code**. You can then start to build out some of the structure. What variables are you going to need? What kinds of logic? 
Knowing where you’re going can help you make fewer mistakes as you’re trying to get there.

Note that for large data sets, the algorithm can take very long to classify because it has to calculate the distance between the test point and every other point in the data!

You can check if your pseudo-code contains all necessary steps afterwards, when scrolling down to "KNN algorithm from scratch" where you find an example of a knn pseudo-code.

## Import and Setup

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# Load data
df = pd.read_csv('data/iris.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
# Relabel column "species"
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["species_encoded"] = le.fit_transform(df["species"])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sepal_length     150 non-null    float64
 1   sepal_width      150 non-null    float64
 2   petal_length     150 non-null    float64
 3   petal_width      150 non-null    float64
 4   species          150 non-null    object 
 5   species_encoded  150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [8]:
# Defining X and y 
x_features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
X = df[x_features]
y = df['species_encoded']

In [9]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
#stratify=y erhält Klassenverteilung

In [ ]:
Train:

Speichere X_train und y_train

Speichere k und ggf. c

Predict für einen Punkt x:

Erzeuge leere Liste distances

Für jeden Trainingspunkt xi in X_train:

berechne d = distance(x, xi, c=c, verbose=False)

speichere (d, label_i) in distances

Sortiere distances nach d aufsteigend

Nimm die ersten k Einträge → k_neighbors

Zähle, welches Label in k_neighbors am häufigsten vorkommt

Gib dieses Label als Vorhersage zurück

## Distance Metrics

As already explained, KNN assigns a class to the test point based on the majority class of  K  nearest neighbors. In general, euclidean distance is used to find nearest neighbors, but other distance metrics can also be used.

As the dimensionality of the feature space increases, the euclidean distance often becomes problematic due to the curse of dimensionality (discussed later).

In such cases, alternative vector-based similarity measures (dot product, cosine similarity, etc) are used to find the nearest neighbors. This transforms the original metric space into one more amicable to point-to-point measurements.

Another distance measure that you might consider is [Mahalanobis distance](https://en.wikipedia.org/wiki/Mahalanobis_distance). Mahalanobis distance attempts to weight features according to their probabilities. On some data sets that may be important.

In general, it's probably a good idea to normalize the data at a minimum. Here's a link to the scikit-learn scaling package: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html . You have to be a little circumspect about employing any technique where the answers change with scaling.

In [15]:
# Implemented own distance function 

import numpy as np

def distance(pt1, pt2, c=2, verbose=True):
    pt1 = np.array(pt1)
    pt2 = np.array(pt2)

    if pt1.shape != pt2.shape:
        raise ValueError("Vectors must have the same length")

    diff = np.abs(pt1 - pt2)
    dist = np.power(np.sum(np.power(diff, c)), 1/c)

    return dist



## KNN Algorithm from scratch


Remember the steps:

1. What information and data does the algorithm need to train and predict the classes of new instances?
This will be the input for our function! 

2. calculate the distance between the test point and each existing data point in the training data.
3. determine the nearest k neighbors.
4. make predictions based on these neighbors.

Hopefully you have already thought of your gameplan, also called pseudo-code. If so, you can compare it to this one:
```
INPUT: X_train, y_train, X_test, k
FOR each object_to_predict in X_test:
    FOR each training_point, index in X_train:
        calculate distance d between object_to_predict and training_point
        store d and index
    SORT distances d in increasing order
    take first k items, get indices of those
    calculate most common class of points at indices in y_train (prediction)
    store prediction
RETURN list of predictions
````

Time to code!
Don't forget that it's good practice to document your own code! This way you can later understand what the purpose of each step was.
Maybe you can even use your pseudo code as documentation :)

In [10]:
import numpy as np
from collections import Counter

class KNNFromScratch:
    def __init__(self, k=5, c=2):
        self.k = k
        self.c = c
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        # "Training" = Daten speichern
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train)

    def predict_one(self, x):
        x = np.array(x)

        # 2) Distanzen zu allen Trainingspunkten
        dists = []
        for xi, yi in zip(self.X_train, self.y_train):
            d = distance(x, xi, c=self.c, verbose=False)
            dists.append((d, yi))

        # 3) k nächste Nachbarn bestimmen
        dists.sort(key=lambda t: t[0])
        k_neighbors = dists[:self.k]

        # 4) Majority Vote
        labels = [label for _, label in k_neighbors]
        most_common_label = Counter(labels).most_common(1)[0][0]
        return most_common_label

    def predict(self, X_test):
        X_test = np.array(X_test)
        return np.array([self.predict_one(x) for x in X_test])

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

knn_scratch = KNNFromScratch(k=5, c=2)
knn_scratch.fit(X_train, y_train)
y_pred = knn_scratch.predict(X_test)

print("Accuracy (scratch):", accuracy_score(y_test, y_pred))


Accuracy (scratch): 0.9777777777777777


## Comparison with sklearn knn implementation

That will be interesting! Check out how your implementation performs in comparison to the one of sklearn!
You can check the confusion matrix and the accuracy score of both algorithms.
If you want, you can check which algorithm is faster!

In [ ]:
# Your code